In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import requests
import pandas as pd
import mysql.connector
from mysql.connector import errorcode

In [5]:
# Spotify
CLIENT_ID = '9189e74191a848bf82c60efbf06229c6'
CLIENT_SECRET = '36c5b1edfa3744e7ac992361fa2b5b02'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
))

generos = ['pop', 'metal', 'reggaeton', 'rap']
años = [2015, 2016]

datos_spotify = []

for genero in generos:
    for ano in años:
        for offset in range(0, 100, 50):
            resultados = sp.search(q=f'genre:"{genero}" year:{ano}',
                                   type='track',
                                   limit=50,
                                   offset=offset)
            for cancion in resultados['tracks']['items']:
                datos_spotify.append({
                    'artista': cancion['artists'][0]['name'],
                    'genero_musical': genero,
                    'nombre': cancion['name'],
                    'ano_lanzamiento': cancion['album']['release_date'][:4],
                    'id': cancion['id']
                })

with open('spotify_datos.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['artista', 'genero_musical', 'nombre', 'ano_lanzamiento', 'id'])
    writer.writeheader()
    writer.writerows(datos_spotify)

print(f"✅ Guardado: spotify_datos.csv ({len(datos_spotify)} canciones)")

# Last.fm
API_KEY_LASTFM = '472626d71b58f9fbdd11821189cb8716'
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

df = pd.read_csv('spotify_datos.csv')
artistas_unicos = list(set(df['artista'].tolist()))

datos_lastfm = []

for artista in artistas_unicos:
    params = {
        "method": "artist.getInfo",
        "artist": artista,
        "api_key": API_KEY_LASTFM,
        "format": "json",
        "lang": "es"
    }
    resp = requests.get(BASE_URL, params=params).json()
    if "artist" in resp:
        bio = resp['artist']['bio']['summary'] if resp['artist'].get('bio') else "No disponible"
        stats = resp['artist'].get('stats', {})
        datos_lastfm.append({
            'artist': artista,
            'biografia': bio.strip(),
            'playcount': stats.get('playcount', 0),
            'oyentes': stats.get('listeners', 0)
        })

with open('lastfm_datos.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['artist', 'biografia', 'playcount', 'oyentes'])
    writer.writeheader()
    writer.writerows(datos_lastfm)

print(f"✅ Guardado: lastfm_datos.csv ({len(datos_lastfm)} artistas)")

✅ Guardado: spotify_datos.csv (800 canciones)
✅ Guardado: lastfm_datos.csv (602 artistas)


In [18]:
# Conexión Python-MySQL.
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',
    database='proyecto_grupal_mod_2'
)

print(cnx) # Muestra la conexión.

In [19]:
mycursor = cnx.cursor()
mycursor.execute("SELECT DATABASE();")
db_actual = mycursor.fetchone()
print("Base de datos activa:", db_actual)

Base de datos activa: ('proyecto_grupal_mod_2',)


In [20]:
df_spotify = pd.read_csv('spotify_datos.csv')
df_lastfm = pd.read_csv('lastfm_datos.csv')

In [21]:
# iniciamos el cursor                               
mycursor = cnx.cursor()
# Lastfm_datos
with open('lastfm_datos.csv', 'r', encoding='utf-8') as archivo_csv:
    reader = csv.DictReader(archivo_csv)
    for fila in reader:
        sql = """
            INSERT INTO artistas (nombre, biografia, playcount, oyentes)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                biografia = VALUES(biografia),
                playcount = VALUES(playcount),
                oyentes = VALUES(oyentes)
        """
        valores = (
            fila['artist'],
            fila['biografia'],
            int(fila['playcount']),
            int(fila['oyentes'])
        )
        mycursor.execute(sql, valores)

print("🎤 Datos insertados/actualizados en `artistas` correctamente")

# Spotify_datos
with open('spotify_datos.csv', 'r', encoding='utf-8') as archivo_csv:
    reader = csv.DictReader(archivo_csv)
    for fila in reader:
        # Buscar id_artista correspondiente
        mycursor.execute(
            "SELECT id_artista FROM artistas WHERE nombre = %s",
            (fila['artista'],)
        )
        resultado = mycursor.fetchone()
        id_artista = resultado[0] if resultado else None

        sql = """
            INSERT INTO canciones (nombre, artista, genero_musical, tipo, año_lanzamiento, id_spotify, id_artista)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        valores = (
            fila['nombre'],
            fila['artista'],
            fila['genero_musical'],
            fila['genero_musical'],
            int(fila['ano_lanzamiento']),
            fila['id'],
            id_artista  # Puede ser NULL si no se encuentra
        )
        mycursor.execute(sql, valores)

print("🎵 Datos insertados en `canciones` correctamente")

# Guardar cambios y cerrar
# cnx.commit()
mycursor.close()
cnx.close()
print("✅ Datos insertados y conexión cerrada.")




🎤 Datos insertados/actualizados en `artistas` correctamente
🎵 Datos insertados en `canciones` correctamente
✅ Datos insertados y conexión cerrada.
